# LSTM 

## Outline
1. Imports
2. Data
3. Model
5. Train
6. Predict

## Imports

In [ ]:
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Data

### Ingestion 

In [ ]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
print("Parent Directory", parentDir)
# Save the path to all of the datasets
india_cases_path = os.path.join(parentDir, "cleaned_datasets/india/daily_cases_india.csv")
india_vacc_path = os.path.join(parentDir, "cleaned_datasets/india/daily_vacc_india.csv")
usa_cases_path = os.path.join(parentDir, "cleaned_datasets/usa/daily_cases_usa.csv")
usa_vacc_path = os.path.join(parentDir, "cleaned_datasets/usa/vacc_usa.csv")

# Quick check to make sure the path exists
print("Path:", india_cases_path)
print("Exists:", os.path.exists(india_cases_path))

In [ ]:
# Load the data as a pandas dataframe
india_cases_df = pd.read_csv(india_cases_path)
india_vacc_df =  pd.read_csv(india_vacc_path)

usa_cases_df = pd.read_csv(usa_cases_path)
usa_vacc_df = pd.read_csv(usa_vacc_path)

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('India Vacc:\n',india_vacc_df.head(),'\n')

print('USA Cases:\n',usa_cases_df.head(),'\n')
print('USA Vacc:\n',usa_vacc_df.head(),'\n')

### Pre-Processing

In [ ]:
# Select only the Confirmed column for univariate analysis
# Selecting from the first index because the 0th index is NaN
india_cases_df = india_cases_df[["Confirmed"]][1:]
usa_cases_df = usa_cases_df[["Confirmed"]][1:]

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

In [ ]:
# Normalize the data
india_cases_mean = india_cases_df.mean()
india_cases_std = india_cases_df.std()

usa_cases_mean = usa_cases_df.mean()
usa_cases_std = usa_cases_df.std()


india_cases_df = (india_cases_df-india_cases_mean)/india_cases_std
usa_cases_df = (usa_cases_df-usa_cases_mean)/usa_cases_std

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

In [ ]:
# Create train test splits
india_cases_train, india_cases_test = train_test_split(india_cases_df, test_size=0.2)
india_vacc_train, india_vacc_test = train_test_split(india_vacc_df, test_size=0.2)

usa_cases_train, usa_cases_test = train_test_split(usa_cases_df, test_size=0.2)
usa_vacc_train, usa_vacc_test = train_test_split(usa_vacc_df, test_size=0.2)

# Visualize splits
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

In [ ]:
# Convert to numpy array
india_cases_train, india_cases_test = india_cases_train.to_numpy().flatten(), india_cases_test.to_numpy()
usa_cases_train, usa_cases_test = usa_cases_train.to_numpy().flatten(), usa_cases_test.to_numpy()

# Visualize outputs
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

In [ ]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
# choose a number of time steps
n_steps = 3
# split into samples
india_cases_train_X, india_cases_train_y = split_sequence(india_cases_train, n_steps)
india_cases_test_X, india_cases_test_y = split_sequence(india_cases_test, n_steps)


# summarize the data
for i in range(0,5):
    print(india_cases_train_X[i], india_cases_train_y[i])

In [ ]:
# Reshape the data into [samples, timesteps, features]
# univariate
n_features = 1

india_cases_train_X = india_cases_train_X.reshape((india_cases_train_X.shape[0], 
                                                   india_cases_train_X.shape[1], n_features))
india_cases_test_X = india_cases_test_X.reshape((india_cases_test_X.shape[0], 
                                                 india_cases_test_X.shape[1], n_features))

# Visualize outputs
print('India Cases:\n',india_cases_train_X,'\n')

## Model 

### Univariate non-stacked

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Define the model
model_uni = Sequential()
model_uni.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model_uni.add(Dense(1))
model_uni.compile(optimizer='adam',loss='mse')
model_uni.summary()

### Univariate stacked 

In [ ]:
# Define the model
model_uni_stacked = Sequential()
model_uni_stacked.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model_uni_stacked.add(LSTM(50, activation='relu'))
model_uni_stacked.add(Dense(1))
model_uni_stacked.compile(optimizer='adam',loss='mse')
model_uni_stacked.summary()

## Train 

In [ ]:
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Univariate non-stacked

In [ ]:
# Fit the model
model_uni.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

### Univariate stacked 

In [ ]:
# Fit the model
model_uni_stacked.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

## Predict

### Univariate stacked 

In [ ]:
x_input = np.array([india_cases_test_X[25]])
print(x_input)
yhat = model_uni.predict(x_input)
print(yhat)

### Univariate stacked 

In [ ]:
x_input = np.array([india_cases_test_X[25]])
print(x_input)
yhat = model_uni_stacked.predict(x_input)
print(yhat)